## Parcial Final

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
import pandas as pd
import scipy.optimize as spo

In [9]:
datos=pd.read_csv('Datos.txt')
lam_30= datos[datos.angle==30]
lam_30

,h,angle,y,sigmay
20,10.0,30.0,0.0013,0.0015
21,20.0,30.0,0.0017,0.0015
22,30.0,30.0,0.0038,0.0015
23,40.0,30.0,0.0059,0.0015
24,50.0,30.0,0.0067,0.0015
25,60.0,30.0,0.0094,0.0015
26,70.0,30.0,0.0122,0.0015
27,80.0,30.0,0.0132,0.0015
28,90.0,30.0,0.0151,0.0015
29,100.0,30.0,0.0186,0.0015


In [17]:
lam_30.h

20     10.0
21     20.0
22     30.0
23     40.0
24     50.0
25     60.0
26     70.0
27     80.0
28     90.0
29    100.0
Name: h, dtype: float64

In [63]:
h_=sym.Symbol('h',real=True)
w_=sym.Symbol('w',real=True)
lamd=sym.Symbol('lambda',real=True)
g_=sym.Symbol('g',real=True)

def Coriolis_sym(h,w,lam,g):
    res=(2*(2**(1/2))/3) *w*sym.cos(lam)*((h**3)/g)**(1/2)
    return res

Coriolis_sym(h_,w_,lamd,g_)

0.942809041582063*w*(h**3/g)**0.5*cos(lambda)

In [64]:
def Coriolis_np(h,w,lam,g):
    res=(2*(2**(1/2))/3) *w*np.cos(lam)*((h**3)/g)**(1/2)
    return res

In [65]:
def chi2(w,h,y,sigmay,angle,g):
    resp=np.sum((y-Coriolis_np(h,w,angle,g))**2 /(2*sigmay**2))
    return resp

e)

In [104]:
h=lam_30.h
y=lam_30.y
sigmay=lam_30.sigmay
angle=lam_30.angle
g=9.81

nll=lambda *w:chi2(*w)
p0=np.ones(10)


In [109]:
minimizar = spo.minimize( nll, p0, args=(h,y,sigmay,angle,g),options={'disp':True} )

minimizar


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 264
         Gradient evaluations: 24


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 8.04507864996885e-08
        x: [ 8.854e-04  4.093e-04  4.981e-04  5.023e-04  4.081e-04
             4.356e-04  4.486e-04  3.973e-04  3.809e-04  4.006e-04]
      nit: 14
      jac: [-5.935e-07  5.354e-07 -2.312e-06 -6.111e-07  1.303e-07
            -1.322e-07  7.259e-07 -7.665e-08  2.781e-07 -3.287e-06]
 hess_inv: [[ 1.044e-06  2.866e-14 ...  9.418e-16 -1.062e-16]
            [ 2.866e-14  1.305e-07 ... -1.605e-17  4.639e-16]
            ...
            [ 9.408e-16 -2.046e-17 ...  1.432e-09  8.547e-18]
            [-1.065e-16  4.631e-16 ...  4.413e-19  1.044e-09]]
     nfev: 264
     njev: 24

In [91]:
minimizar.fun/(10-1)

0.12730173386282867

f)

In [108]:
_x = np.linspace(np.min(h),np.max(h),100)
bestparam=minimizar.x
_y = Coriolis_np(_x,bestparam)

#plt.errorbar(x,y,yerr=sigma,fmt='.')
plt.plot(_x,_y,color='r')

TypeError: Coriolis_np() missing 2 required positional arguments: 'lam' and 'g'